## Connection to Thymio

In [1]:
# Connect to Thymio with ClientAsync
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()

# Lock Thymio
aw(node.lock())

Node fe36b4a4-2452-4ab5-8d68-b28bb42f53d5

## Importation of libraries

In [2]:
from kalman_filtering.kalman_filter import *
from thymio.control import *
from computer_vision.vision import *
from path_planning.path_plan import *
from navigation.collision_avoidance import *

## Main script

In [3]:

u_old = 0
mu_predict_old = 0
cov_predict_old = 0
view_blocked = False
active = True
end = False
start = True
path = None
segment_index = 0
obstacle = 0
error_est=[10,10]

map = Map()
if P_VISION:
    print(P_SETUP)
    print("Place the Thymio and the destination, check the obstacles and the corners. Then, press S.")

while True:
    map.pre_update()
    cv.waitKey(0)
    if cv.waitKey(20) & 0xFF==ord('s'):
        if map.get_found_corners():
            map.update()
            break
        elif P_VISION:
            print("USER: Must find corners before start")

if P_VISION: print(P_START)
if P_VISION: map.info()
await node.wait_for_variables()

while active == True:
    active = await stop_thymio(node)

    map.update()
    
    # State variables measurement
    # await node.wait_for_variables()
    # vel_meas = velocity_with_odometry(node, map.robot[2])

    # Filtering
    # y = np.array([map.robot[0]], [map.robot[1]], [vel_meas[0]], [vel_meas[1]])
    # x_est, cov_est = kalman_filter(y, u_old, mu_predict_old, cov_predict_old, view_blocked)
    # pose_est = x_est[0:2]
    # pose_est.append(map.robot[2])
    # error_est = cov_est[0:2]
    # error_est.append(0)

    # Global Navigation
    #create path
   
    # Global Navigation
    if start == True:
        ex_path, corners = compute_visibility_matrix(map.robot[0:2], map.destination[0:2], map.obstacles)
        map.obstacles_lines = possible_lignes(ex_path, corners)
        map.target_lines = a_star_search(corners, ex_path)
        path = path_functions(map.target_lines)
        start = False

    #local nav
    u, obstacle, segment = await collision_avoidance(path, segment_index, map.robot, node) # Zacharie


    if not obstacle:
        u, segment_index, end = path_direction(map.robot, path, segment_index)

    await node.set_variables(motors(int(u[0]), int(u[1])))

    # u_old = u
    # cov_predict_old = cov_est
    # mu_predict_old = x_est
    #obstacle_old = obstacle

    if cv.waitKey(20) & 0xFF==ord('p'):
        map.vision_stop()
        break

=====[   SETUP      ]===============
Place the Thymio and the destination, check the obstacles and the corners. Then, press S.
=====[   START      ]===============
=====[   MAP INFO   ]===============
* FOUND CORNERS: True

* FOUND ROBOT: True
Position: [584, 323]
Orientation: -2.8484067399272353

* FOUND DESTINATION: True
Position: [39, 126]

* OBSTACLES: 2

Path_reached
Path_reached
Path_reached
Path_reached
Path_reached
Path_reached
Path_reached
Path_reached
Path_reached
Path_reached
Path_reached
Path_reached
Path_reached
Path_reached
Path_reached
ERROR: Could not detect Destination
Path_reached
ERROR: Could not detect Destination
Path_reached
ERROR: Could not detect Destination
Path_reached
ERROR: Could not detect Destination
Path_reached
ERROR: Could not detect Destination
Path_reached
ERROR: Could not detect Destination
Path_reached
ERROR: Could not detect Destination
Path_reached
ERROR: Could not detect Destination
Path_reached
ERROR: Could not detect Destination
Path_reached
Pa

ValueError: not enough values to unpack (expected 3, got 2)

## Motors control test

In [ ]:
active = True
vel_meas = []

while active:
    await node.wait_for_variables()
    #vel_meas.append(velocity_with_odometry(node, 0)[0])
    await node.set_variables(motors(-100, 100))
    active = await stop_thymio(node)

await node.set_variables(motors(0, 0)) 

vel_meas_mean = np.mean(vel_meas[-100:])
vel_meas_var = np.var(vel_meas[-100:])
print("Velocity measured:", vel_meas)
print("Velocity mean:", vel_meas_mean)
print("Velocity variance:", vel_meas_var)

Center button pressed. Stopping the algorithm.
Velocity measured: []
Velocity mean: nan
Velocity variance: nan


C:\Users\BossTala\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\_core\fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\BossTala\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\BossTala\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\_core\fromnumeric.py:4318: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\BossTala\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\_core\_methods.py:184: Runt

In [ ]:
node.var

{'_fwversion': [14, 0],
 '_id': [-27410],
 '_imot': [-152, -234],
 '_integrator': [-978, -1292],
 '_productId': [8],
 '_vbat': [714, 711],
 'acc': [-1, 1, 20],
 'acc._tap': [32],
 'button.backward': [0],
 'button.center': [1],
 'button.forward': [0],
 'button.left': [0],
 'button.right': [0],
 'buttons._mean': [10649, 9158, 13699, 13651, 15054],
 'buttons._noise': [76, 61, 82, 92, 102],
 'buttons._raw': [10646, 9153, 12851, 13651, 15043],
 'event.args': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'event.source': [-27410],
 'leds.bottom.left': [0, 0, 0],
 'leds.bottom.right': [0, 0, 0],
 'leds.circle': [0, 0, 0, 0, 0, 0, 0, 0],
 'leds.top': [0, 0, 0],
 'mic._mean': [258],
 'mic.intensity': [106],
 'mic.threshold': [0],
 'motor.left.pwm': [641],
 'motor.left.speed': [168],
 'motor.left.target': [-100],
 'motor.right.pwm': [-12],
 'motor.right.speed': [203],
 'motor.right.t